In [1]:
import pandas as pd
import numpy as np
import os
import time
import openpyxl
from io import StringIO
import warnings
warnings.filterwarnings('ignore')

import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

----
- excel 불러오기 + 회사이름 표시

In [8]:
data_path = './'
file_name = 'Korea_GTDB.xlsx'

In [11]:
os.path.join(data_path, file_name)

'./Korea_GTDB.xlsx'

In [ ]:
pd.read_excel()

In [9]:
def preprocess_data(path):
    data = pd.read_excel(path, sheet_name=None) #모든 sheet 불러오기
    data.pop('exmaple') #필요없는 시트 삭제
    data.pop('템플릿')
    data.pop('setting') 
    data.pop('front')

    
    '''
    merge all the excel sheets to dataframe
    '''
    df = pd.DataFrame()
    sheets = list(data.keys())

    for i in range(len(sheets)):
        sheet = data[sheets[i]]
        df = df.append(sheet)
        df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]

        
        
    # 컬럼 'Company'에 시트이름 가져와서 회사명채우기
    company_name = list(data.keys()) 
    rep_times = []
    fill_company=[]

    for i in range(len(company_name)):
        sheet = data[company_name[i]]
        company_length = len(sheet) #company 마다 컬럼 개수 구하기
        rep_times.append(company_length)
        
        company_repeat = np.repeat(company_name[i], rep_times[i], axis=0) #데이터프레임에 회사 이름 채우기 위해 회사 goal 길이만큼 반복
        fill_company.append(company_repeat)

    fill_company = [i for inner_list in fill_company for i in inner_list] #리스트 차원 낮추기

    df = df.assign(Company = fill_company) #회사이름 assign 시키기
    df = df[df['Goal'].notna()]
    df.reset_index(drop=True, inplace=True)
    
    return df

In [6]:
#동시출력 가능코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [12]:
%%time
df = preprocess_data(os.path.join(data_path, file_name))
df.shape
df

CPU times: user 19.2 s, sys: 766 µs, total: 19.2 s
Wall time: 19.2 s


,Company,Page,Goal,Metric,Focus,Target(목표치),Target_Type,Goal_Direction,Due(목표년도),Result(결과),Subtheme,Baseline_Year,Source year,is_goal
0,018880_한온시스템,14.0,"2022년 친환경차 부문의 매출 비중은 26%에 달할 것으로 예상되며, 2025년에...",%,친환경 상품 판매/개발,40.0,REL,증가,2025.0,NaN,NaN,NaN,2021.0,NaN
1,018880_한온시스템,17.0,한온시스템은 2040년 탄소중립 달성을 목표로 2030년까지 2019년 기준 대비 ...,%,온실가스 배출,50.0,REL,감소,2030.0,NaN,NaN,NaN,2021.0,NaN
2,018880_한온시스템,20.0,에너지 사용량 (단위: MWh),KWh,에너지 사용,NaN,ABS,감소,NaN,146032000.0,NaN,NaN,2021.0,NaN
3,018880_한온시스템,22.0,2021년 온실가스 배출량(Scope 1 및 Scope 2)은 267천 tCO2eq...,tCO₂eq,온실가스 배출,NaN,ABS,감소,2021.0,267000.0,NaN,2019.0,2021.0,NaN
4,018880_한온시스템,23.0,"용수 사용량(단위: 1,000m3)",㎡(천),용수 사용,NaN,ABS,감소,NaN,1419.0,NaN,NaN,2021.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3711,035250_강원랜드,44.0,프로젝트의\n목표는 2019년부터 2025년까지 7년간 총 21개의 청년 창업기업을...,개,사회 공헌 프로그램,21.0,ABS,증가,2025.0,NaN,NaN,2019.0,2021.0,NaN
3712,035250_강원랜드,48.0,"한편, 청년일자리 목표 달성을 위한 명예퇴직을 임단협 합의로 매년\n상반기 정기 시...",명,신입 사원 채용,NaN,ABS,증가,NaN,7091.0,NaN,2021.0,2021.0,NaN
3713,035250_강원랜드,50.0,KL 인권경영지표,점,ESG 관리,NaN,ABS,증가,NaN,85.0,NaN,2021.0,2021.0,NaN
3714,035250_강원랜드,54.0,반부패 청렴교육 이수율,%,윤리·준법 교육,NaN,REL,증가,NaN,54.0,NaN,2021.0,2021.0,NaN


In [13]:
df['Company'].unique()

array(['018880_한온시스템', '000270_기아', '017810_풀무원', '002380_KCC',
       '096770_SK이노베이션', '034730_SK', '003670_포스코케미칼', '010950_S-Oil',
       '000120_CJ대한통운', '004990_롯데지주', '23530_롯데쇼핑', '078930_GS',
       '015760_한국전력공사', '090430_아모레퍼시픽', '033780_KT&G', '006360_GS건설',
       '034020_두산중공업', '011790_SKC', '010060_OCI', '298020_효성티앤씨',
       '004800_효성', '298040_효성중공업', '298050_효성첨단소재', '298000_효성화학',
       '001740_SK네트웍스', '010120_LSELECTRIC', '285130_SK케미칼',
       '017670_SK텔레콤', '000660_SK하이닉스', '030200_KT', '036460_한국가스공사',
       '282330_BGF리테일', '329180_현대중공업', '009540_한국조선해양', '000810_삼성화재',
       '005930_삼성전자', '267250_HD현대', '128940_한미약품', '000720_현대건설',
       '006400_삼성SDI', '003410_쌍용C&E', '016360_삼성증권', '097950_CJ제일제당',
       '021240_코웨이', '047040_대우건설', '005380_현대차', '005490_POSCO홀딩스',
       '086790_하나금융지주', '066570_LG전자', '051900_LG생활건강', '034220_LG디스플레이',
       '051910_LG화학', '000100_유한양행', '011200_HMM', '003490_대한항공',
       '032640_LG유플러스', '011170_롯데케미칼', '01

-----

In [14]:
def mmr(doc, top_n, diversity):

    okt = Okt() #한국어 형태소 분석기 okt 

    tokenized_doc = okt.pos(doc)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun']) #형태소 명사 추출


    n_gram_range = (1, 2) #2개의 gram이 나오도록 설정 -> (1,3) 했을때 보단 나은듯
    
    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()

    model = SentenceTransformer('distilbert-base-nli-mean-tokens')
    doc_embedding = model.encode([doc])
    candidate_embeddings = model.encode(candidates)


    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding) #키워드 간의 유사도 리스트
    word_similarity = cosine_similarity(candidate_embeddings) # 각 키워드들 간의 유사도


    
    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    candidates_idx = [i for i in range(len(candidates)) if i != keywords_idx[0]]

    
    for _ in range(top_n - 1):
        '''
        최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
        ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
        '''
        
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        
        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [candidates[idx] for idx in keywords_idx]

----

- 실험1

In [18]:
df_top1 = df.copy()
df_top1["Focus_keybert"]= pd.Series()
df_top1 = df_top1[['Company', 'Page', 'Goal', 'Metric', 'Focus_keybert','Focus', 'Target(목표치)',
       'Target_Type', 'Goal_Direction', 'Due(목표년도)', 'Result(결과)', 'Subtheme',
       'Baseline_Year', 'Source year', 'is_goal']]
df_top1

,Company,Page,Goal,Metric,Focus_keybert,Focus,Target(목표치),Target_Type,Goal_Direction,Due(목표년도),Result(결과),Subtheme,Baseline_Year,Source year,is_goal
0,018880_한온시스템,14.0,"2022년 친환경차 부문의 매출 비중은 26%에 달할 것으로 예상되며, 2025년에...",%,NaN,친환경 상품 판매/개발,40.0,REL,증가,2025.0,NaN,NaN,NaN,2021.0,NaN
1,018880_한온시스템,17.0,한온시스템은 2040년 탄소중립 달성을 목표로 2030년까지 2019년 기준 대비 ...,%,NaN,온실가스 배출,50.0,REL,감소,2030.0,NaN,NaN,NaN,2021.0,NaN
2,018880_한온시스템,20.0,에너지 사용량 (단위: MWh),KWh,NaN,에너지 사용,NaN,ABS,감소,NaN,146032000.0,NaN,NaN,2021.0,NaN
3,018880_한온시스템,22.0,2021년 온실가스 배출량(Scope 1 및 Scope 2)은 267천 tCO2eq...,tCO₂eq,NaN,온실가스 배출,NaN,ABS,감소,2021.0,267000.0,NaN,2019.0,2021.0,NaN
4,018880_한온시스템,23.0,"용수 사용량(단위: 1,000m3)",㎡(천),NaN,용수 사용,NaN,ABS,감소,NaN,1419.0,NaN,NaN,2021.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3711,035250_강원랜드,44.0,프로젝트의\n목표는 2019년부터 2025년까지 7년간 총 21개의 청년 창업기업을...,개,NaN,사회 공헌 프로그램,21.0,ABS,증가,2025.0,NaN,NaN,2019.0,2021.0,NaN
3712,035250_강원랜드,48.0,"한편, 청년일자리 목표 달성을 위한 명예퇴직을 임단협 합의로 매년\n상반기 정기 시...",명,NaN,신입 사원 채용,NaN,ABS,증가,NaN,7091.0,NaN,2021.0,2021.0,NaN
3713,035250_강원랜드,50.0,KL 인권경영지표,점,NaN,ESG 관리,NaN,ABS,증가,NaN,85.0,NaN,2021.0,2021.0,NaN
3714,035250_강원랜드,54.0,반부패 청렴교육 이수율,%,NaN,윤리·준법 교육,NaN,REL,증가,NaN,54.0,NaN,2021.0,2021.0,NaN


In [16]:
from tqdm import tqdm
from time import sleep

In [ ]:
%%time

for i in tqdm(range(len(df_top1))):
    mmr_input = mmr(df_top1['Goal'][i], top_n = 1, diversity=0.2)
    df_top1.loc[i, 'Focus_keybert'] = mmr_input

  6%|██▍                                     | 227/3716 [03:39<56:59,  1.02it/s]

In [ ]:
#제대로 뽑혔나 확인
df_top1


---
- 실험2) 유사도 높은 상위 2개 추출하기(top_n=2)

In [ ]:
df_top2 = df.copy()
df_top2["Focus_keybert"]= pd.Series()
df_top2 = df_top2[['Company', 'Page', 'Goal', 'Metric', 'Focus_keybert','Focus', 'Target(목표치)',
       'Target_Type', 'Goal_Direction', 'Due(목표년도)', 'Result(결과)', 'Subtheme',
       'Baseline_Year', 'Source year', 'is_goal']]

In [ ]:
%%time

for i in tqdm(range(len(df_top1))):
    mmr_input = mmr(df_top1['Goal'][i], top_n = 2, diversity=0.2)
    df_top1.loc[i, 'Focus_keybert'] = mmr_input